In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
import itertools
from string import punctuation
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
import pickle
# from multilabel_data_handler import get_data

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from collections import Counter

In [3]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import neighbors
from sklearn import ensemble
from sklearn import neural_network
from sklearn import linear_model
import os

/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
# file used to write preserve the results of the classfier
# confusion matrix and precision recall fscore matrix

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    return plt

In [5]:
##saving the classification report
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='macro'))
    avg.append(accuracy_score(y_true, y_pred, normalize=True))
    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support','accuracy']
    list_all=list(metrics_summary)
    list_all.append(cm.diagonal())
    class_report_df = pd.DataFrame(
        list_all,
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-2] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T

In [6]:
from commen_preprocess import *

....start....cleaning


In [7]:
eng_train_dataset = pd.read_csv('../AMI@EVALITA2018/en_training.tsv', sep='\t')
eng_test_dataset = pd.read_csv('../AMI@EVALITA2018/en_testing.tsv', sep='\t')
eng_train_dataset = eng_train_dataset.sample(frac=1).reset_index(drop=True)


In [8]:
eng_train_dataset

id                                               text  misogynous  \
0      140  This pussy mine girl tell me you like Daddy di...           1   
1     2824  When your wingman gets bunned up and leaves yo...           0   
2     1812        @glaiveXD WHERE THE FUCK AM I U STUPID CUNT           0   
3      349  When you join a relationship women leave this ...           1   
4      273  Twinkle twinkle little slut, you've got just w...           1   
5      593  Bitch I know you are not there https://t.co/G4...           1   
6     3145  Me flirting- *pulls phone out* 'Look at my dic...           0   
7     1577  It's not rape if women are considered objects!...           1   
8     3499  RT MightyBusterBro: MOST HATED WOMAN Hysterica...           0   
9     2914  I just got a ticket but in my defense I didn't...           0   
10    3047  Friend- I'm getting married Me- https://t.co/h...           0   
11    3500  @WithMyPrez4Ever It’s hysterical! If I were th...           0   
12    3983  Conte being a little bitch as per. Get sacked ...           0   
13    1000  @aboyseth mermaids are women. Therefore they s...           1   
14    2753  Her- you usually wear condoms tho, right? Me- ...           0   
15    3498  #nowPlaying Regain Your Poise Hysterical Woman...           0   
16     674  @ImCardiB being a empowering woman type a bitc...           1   
17      33  also this girl tried to twerk me away from bar...           1   
18     621  @AnngelicaaPerez Bitch come out you love to ta...           1   
19    1345  Dear Akkorou, Bitch ass nigga cunt on a stick ...           1   
20    3089  RT @themrsqueenbee: I'm just a materialistic n...           0   
21    3105  Idk who these things are or why they're popula...           0   
22     113  lil is beautiful, gays are cute, i'm very stro...           1   
23    3362  Neter Sesen Book 2 Verses 61 -62: Tehuti: 'Do ...           0   
24    2811  People that hate on you that you've done nothi...           0   
25    1682  1 hoe 2 hoe 3 hoe 4, girl you know you're just...           1   
26    2775  RT @ggwbt: 😬 #goodgirlwithbadthoughts 💅🏽 https...           0   
27     833  What's the difference between your wife and yo...           1   
28     494  I'm back in it bitch If the head game is right...           1   
29    2542  When you're trying to figure out if you have e...           0   
...    ...                                                ...         ...   
3970  1006  @Carflyy she's a woman. She can't help the fac...           1   
3971  2165  A middle aged woman just shouted 'well done' a...           0   
3972  2702  When someone tries to take the last piece of p...           0   
3973   386  @BasicMormon So do we! Check out our account f...           1   
3974  1849  Sometimes you gotta beat ya dick like it stole...           0   
3975  3588  Waking up Sunday morning hungover as fuck like...           0   
3976  2869                          It must suck to not be me           0   
3977  1409  Y’all expect women to settle for trash treatme...           1   
3978  1564  @DrPhil you mean she still has morals after wh...           1   
3979  1192  @SantinaDiMaggio happy birthday ya lil freak! ...           1   
3980   830  What does a woman and a tampon have in common?...           1   
3981   590         What a stupid cunt https://t.co/Wv1rVaId2D           1   
3982   211  @AmericansRDumb1 dont u have anything better t...           1   
3983  1838  Ahhh my cousin's fetus doesn't have a dick and...           0   
3984   855  When you finally get the bitch to leave https:...           1   
3985  2123  you made yourself a bed at the bottom of the b...           0   
3986  2594           Flirting in 2016 https://t.co/6K9rVnYZjA           0   
3987  4038  @jojolang9 @KurtSchlichter Congratulations, Jo...           0   
3988   484  What happens when you drug, rape and sodomize ...           1   
3989  3175  If you can't spell 'masturbate' right, we can'...           0   
3990

In [9]:
eng_train_dataset['misogyny_category'].value_counts()

0                    2215
discredit            1014
sexual_harassment     352
stereotype            179
dominance             148
derailing              92
Name: misogyny_category, dtype: int64

In [10]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import os
GLOVE_MODEL_FILE = "../../LEAM-master/glove.twitter.27B/glove.twitter.27B.200d.txt"
# GLOVE_MODEL_FILE="../../../glove.840B.300d.txt"
print(os.path.isfile(GLOVE_MODEL_FILE))




import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    i=0
    for line in f:
        i=i+1
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
        if(i%10000==0):
            print("count"+str(i))
    print("Done.",len(model)," words loaded!")
    return model



EMBEDDING_DIM = 200


# glove_file = GLOVE_MODEL_FILE
# tmp_file = get_tmpfile("test_crawl_300.txt")

# # call glove2word2vec script
# # default way (through CLI): python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>
# from gensim.scripts.glove2word2vec import glove2word2vec
# glove2word2vec(glove_file, tmp_file)


# # In[29]:


# word2vec_model = KeyedVectors.load_word2vec_format(tmp_file)
word2vec_model = loadGloveModel(GLOVE_MODEL_FILE)

True
Loading Glove Model
count10000
count20000
count30000
count40000
count50000
count60000
count70000
count80000
count90000
count100000
count110000
count120000
count130000
count140000
count150000
count160000
count170000
count180000
count190000
count200000
count210000
count220000
count230000
count240000
count250000
count260000
count270000
count280000
count290000
count300000
count310000
count320000
count330000
count340000
count350000
count360000
count370000
count380000
count390000
count400000
count410000
count420000
count430000
count440000
count450000
count460000
count470000
count480000
count490000
count500000
count510000
count520000
count530000
count540000
count550000
count560000
count570000
count580000
count590000
count600000
count610000
count620000
count630000
count640000
count650000
count660000
count670000
count680000
count690000
count700000
count710000
count720000
count730000
count740000
count750000
count760000
count770000
count780000
count790000
count800000
count810000
count820000


In [11]:
### stopwords and punctuations are not removed but text is cleaned and stemmed
def glove_tokenize_norem(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    words = text.split()
    words =[ps.stem(word) for word in words]
    return words

####stopwords and punctuations are removed along with that text is cleaned ans stemmed
def glove_tokenize(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    words =[ps.stem(word) for word in words]
    return words
def glove_tokenize_embed(text):
    #text = tokenizer(text)
    text=clean(text, remove_stopwords=False, remove_punctuations=False)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    words = [word for word in words if word not in STOPWORDS]
    return words

In [12]:
def convert_class_label(input_text):
    if input_text==1:
        return 'misogyny'
    elif input_text==0:
        return 'non-misogyny'
    else:
        print('Wrong Input', input_text)
        sys.exit(0)

In [13]:
# pd_train = pd.DataFrame(columns=['id','misogynous','text'])
eng_train_dataset["text"].replace('', np.nan, inplace=True)
eng_train_dataset.dropna(subset=['text'], inplace=True)

pd_train_binary = eng_train_dataset[['id','misogynous','text','misogyny_category','target']]
pd_train_category = eng_train_dataset[['id','misogynous','text','misogyny_category']]
pd_train_target = eng_train_dataset[['id','misogynous','text','target']]
pd_test = eng_test_dataset[['id','text']]

pd_train_category = pd_train_category.loc[pd_train_category['misogynous'] == 1]
pd_train_target = pd_train_target.loc[pd_train_target['misogynous'] == 1]
pd_train_target.drop(['misogynous'], axis=1)                                      
pd_train_category.drop(['misogynous'], axis=1)                                      

# pd_train['class'] =pd_train.apply(lambda row: convert_class_label(row['misogynous']), axis=1)

pd_train_binary['class'] = pd_train_binary['misogynous']
pd_train_category['class'] = pd_train_category['misogyny_category']
pd_train_target['class'] = pd_train_target['target']

# for count, each in enumerate(train_data):
#     try:
#         pd_train.loc[count]  = [each['id'], convert_class_label(each['CounterSpeech']), each['Community'],each['Category'],each['commentText']]
#     except:
#         pass
print('Training Data Loading Completed...')

Training Data Loading Completed...


/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
def get_data(pd_train):
    comments=pd_train['text'].values
    labels=pd_train['class'].values
    list_comment=[]
    for comment,label in zip(comments,labels):
        temp={}
        temp['text']=comment
        temp['label']=label
        list_comment.append(temp)
    return list_comment 

def get_data_test(pd_test):
    comments=pd_test['text'].values
    list_comment=[]
    for comment in comments:
        temp={}
        temp['text']=comment
        list_comment.append(temp)
    return list_comment 


In [15]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=12,
                       allow_soft_placement=True, device_count = {'CPU': 12})

def get_embeddings(messages):
      
    with tf.Session(config=config) as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            message_emb = session.run(embed(messages))
            
    print("ending")
    return np.array(message_emb)

/home/binny/anaconda3/envs/punyajoy-nogpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_11
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_12:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/v

In [16]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

TOKENIZER = glove_tokenize
#google encoding used where text is cleaned  
def gen_data_google(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        #X.append(tokenizer(comment['text']))
        X.append(clean(comment['text'], remove_stopwords=True, remove_punctuations=True))
    #TFIDF_feature = 'bpe_text'

    #Word Level Features
    X =get_embeddings(X)
    # print y
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

#google encoding used where text is not cleaned 
def gen_data_google2(data):
    comments = get_data(data)
    X, y = [],[]
    for comment in comments:
        y.append(comment['label'])
        X.append(clean(comment['text'], remove_stopwords=False, remove_punctuations=False))
    #Word Level Features
    X =get_embeddings(X)
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X,y

### tfidf feature generation was used here where stopwords and punctuations are removed 
def gen_data_new_tfidf(data):
    comments = get_data(data)
    comments_test=get_data_test(pd_test)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])

    X1=[]
    for comment in comments_test:
        X1.append(comment['text'])


    #Word Level Features
    word_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3),
                min_df=1, 
                strip_accents='unicode',
                #smooth_idf=1,
                analyzer='word', 
                stop_words='english',
                tokenizer=TOKENIZER,             
                max_features=5000)
    
    
    #charlevel features new
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    #stop_words='english',
    ngram_range=(2, 6),
    max_features=10000)
    word_vectorizer.fit(X+X1)
    char_vectorizer.fit(X+X1)
    
    with open('tfidf_word_vectorizer.pk', 'wb') as fout:
         pickle.dump(word_vectorizer,fout)

    with open('tfidf_char_vectorizer.pk', 'wb') as fout:
        pickle.dump(char_vectorizer,fout)
    
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    return X, y

### tfidf feature generation was used here where stopwords and punctuations are not removed 
def gen_data_new_tfidf2(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])


    #Word Level Features
    word_vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3),
                min_df=1, 
                strip_accents='unicode',
                #smooth_idf=1,
                analyzer='word', 
                #stop_words='english',
                tokenizer=glove_tokenize_norem,             
                max_features=5000)
    
    
    #charlevel features new
    char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    #stop_words='english',
    ngram_range=(2, 6),
    max_features=10000)
    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    
    with open('tfidf_word_vectorize_noclean.pk', 'wb') as fout:
         pickle.dump(word_vectorizer,fout)

    with open('tfidf_char_vectorizer_noclean.pk', 'wb') as fout:
         pickle.dump(char_vectorizer,fout)
        
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

## combination of not cleaned google encodings and tfidf features where stopwords and punctuations are not removed 
def combine_tf_google_rem(data):
    X,_=gen_data_google(data)
    X1,y=gen_data_new_tfidf(data)
#     X1,y=gen_data_old_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

## combination of cleaned google encodings and tfidf features where stopwords and punctuations are ssremoved 
def combine_tf_google_norem(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf2(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def combine_tf_rem_google_norem(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def combine_tf_norem_google_rem(data):
    X,_=gen_data_google(data)
    X1,y=gen_data_new_tfidf2(data)
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

def gen_data_embed(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        words = glove_tokenize(comment['text'].lower())
        emb = np.zeros(EMBEDDING_DIM)
        for word in words:
            try:
                emb += word2vec_model[word]
            except:
                pass
        if len(words)!=0:
            emb /= len(words)
        X.append(emb)
        y.append(comment['label'])

    return X, y


def combine_tf_rem_google_norem_embed(data):
    X,_=gen_data_google2(data)
    X1,y=gen_data_new_tfidf(data)
    X2,_=gen_data_embed(data)
    X=np.concatenate((np.array(X), np.array(X1),np.array(X2)), axis=1)
    return X,y



###old tfidf

def gen_data_old_tfidf(data):
    comments = get_data(data)
    X, y = [], []
    for comment in comments:
        y.append(comment['label'])
        X.append(comment['text'])
    with open('../tfidf_word_vectorizer.pk', 'rb') as fin:
        word_vectorizer = pickle.load(fin)

    with open('../tfidf_char_vectorizer.pk', 'rb') as fin:
        char_vectorizer = pickle.load(fin)


    
    word_vectorizer.fit(X)
    char_vectorizer.fit(X)
    
    test_word_features = word_vectorizer.transform(X)
    test_char_features = char_vectorizer.transform(X)
    X = list(hstack([test_char_features, test_word_features]).toarray())
    #y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    
    return X, y

def select_comments_whose_embedding_exists(flag):
    # selects the comments as in mean_glove_embedding method
    # Processing
    comments = get_data(flag)
    X, Y = [], []
    comment_return = []
    for tweet in comments:
        #print(tweet)
        _emb = 0
        words = TOKENIZER(tweet['text'].lower())
        for w in words:
            #print(w)
            if w in word2vec_model and w is not None:  # Check if embeeding there in GLove model
                _emb+=1
        if _emb:   # Not a blank tweet
            comment_return.append(tweet)
    print('Comments selected:', len(comment_return))
    return comment_return

def gen_data():
    comments = select_comments_whose_embedding_exists(0)
    X, y = [], []
    for comment in comments:
        words = glove_tokenize(comment['text'].lower())
        emb = numpy.zeros(EMBEDDING_DIM)
        for word in words:
            try:
                emb += word2vec_model[word]
            except:
                pass
        emb /= len(words)
        X.append(emb)
        y.append(comment['label'])

    # print y
    y = MultiLabelBinarizer(classes = (1,2,3,4,5,6,7,8,9,10)).fit_transform(y)
    print
    return X, y

## combination of not cleaned google encodings and tfidf features where stopwords and punctuations are not removed 
def combine_tf_google_glove_rem():
    X,_=gen_data_google()
    X1,y=gen_data_new_tfidf()
#     X1,y=gen_data_old_tfidf()
    X=np.concatenate((np.array(X), np.array(X1)), axis=1)
    return X,y

In [17]:
from imblearn.over_sampling import ADASYN
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
def get_model(m_type=None):
    if not m_type:
        print("ERROR: Please specify a model type!")
        return None
    if m_type == 'decision_tree_classifier':
        logreg = tree.DecisionTreeClassifier(class_weight='balanced')
    elif m_type == 'gaussian':
        logreg = GaussianNB()
    elif m_type == 'logistic_regression':
        logreg = LogisticRegression(class_weight='balanced',n_jobs=10, random_state=42)
    elif m_type == 'MLPClassifier':
#         logreg = neural_network.MLPClassifier((500))
        logreg = neural_network.MLPClassifier((500),random_state=42,early_stopping=False)
    elif m_type == 'KNeighborsClassifier':
#         logreg = neighbors.KNeighborsClassifier(n_neighbors = 10)
        logreg = neighbors.KNeighborsClassifier()
    elif m_type == 'ExtraTreeClassifier':
        logreg = tree.ExtraTreeClassifier()
    elif m_type == 'ExtraTreeClassifier_2':
        logreg = ensemble.ExtraTreesClassifier()
    elif m_type == 'RandomForestClassifier':
        logreg = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=3)
    elif m_type == 'SVC':
        logreg = LinearSVC(class_weight='balanced');
    elif m_type == 'Catboost':
        logreg = CatBoostClassifier(use_best_model=False,iterations=1000, random_state=42, loss_function='MultiClass',class_weights=[1,3,9,9,9])
#         logreg = CatBoostClassifier(scale_pos_weight=0.8, random_seed=42,);
    elif m_type == 'XGB_classifier':
#         logreg=XGBClassifier(silent=False,eta=0.1,objective='binary:logistic',max_depth=5,min_child_weight=0,gamma=0.2,subsample=0.8, colsample_bytree = 0.8,scale_pos_weight=1,n_estimators=500,reg_lambda=3,nthread=12)
        logreg=XGBClassifier(silent=False,objective='binary:logistic',scale_pos_weight=0.8,reg_lambda=3,nthread=12, random_state=42)
    elif m_type == 'binny_test':
        clf1 = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=6,max_features='auto')
        clf2 = tree.DecisionTreeClassifier(random_state=42, class_weight='balanced',max_depth=6)
        clf3 = LogisticRegression(class_weight='balanced',penalty="l2",C=0.1, dual=True, random_state=42, n_jobs=3)
        clf4 = XGBClassifier(silent=False,objective='binary:logistic',scale_pos_weight=0.8,reg_lambda=3,nthread=12, random_state=42)
        est_list = [('lr', clf1), ('rf', clf2), ('gnb', clf3), ('xgb', clf4)]
        logreg = ensemble.VotingClassifier(est_list,voting='soft',n_jobs=6)
    else:
        print("give correct model")
    print(logreg)
    return logreg

def get_feature(f_type=None,data=None):
    if not f_type:
        print("ERROR: Please specify a model type!")
        return None
    if f_type == 'google_not_preprocess':
        X,y=gen_data_google2(data)
    elif f_type == 'word_to_vec_embed':
        X,y=gen_data_embed(data)
    elif f_type == 'google_preprocess':
        X,y=gen_data_google(data)
    elif f_type == 'tfidf_not_preprocess':
        X,y=gen_data_new_tfidf2(data)
    elif f_type == 'tfidf_preprocess':
        X,y=gen_data_new_tfidf(data)
    elif f_type == 'google_preprocess_tfidf_preprocess':
        X,y=combine_tf_google_rem(data)
    elif f_type == 'google_nopreprocess_tfidf_nopreprocess':
        X,y=combine_tf_google_norem(data)
    elif f_type == 'google_preprocess_tfidf_nopreprocess':
        X,y=combine_tf_norem_google_rem(data)
    elif f_type == 'google_nopreprocess_tfidf_preprocess':
        X,y=combine_tf_rem_google_norem(data)
    elif f_type == 'google_nopreprocess_tfidf_preprocess_embed':
        X,y=combine_tf_rem_google_norem_embed(data)
    else:
        print("give correct feature selection")    
    print(f_type)
    return X,y



In [25]:
# from imblearn.combine import SMOTETomek 
from sklearn.metrics import accuracy_score
import joblib
from sklearn.model_selection import StratifiedKFold as skf

def binny_classifier_run(X,y,model,model_name,label_map,img_name,report_name,save_model=False):
    Classifier_Train_X = np.array(X, copy=False)
    Classifier_Train_Y = y
  
    temp=[]
    for data in Classifier_Train_Y:
        temp.append(label_map[data])
#     print(Classifier_Train_Y)
    Classifier_Train_Y=np.array(temp)
    
    model_featureSelection = SelectFromModel(ensemble.RandomForestClassifier(n_estimators=50, class_weight='balanced', 
                                                                                     n_jobs=12, max_depth=3))
    print('Before Num features=',Classifier_Train_X.shape[1], Counter(Classifier_Train_Y))
    Classifier_Train_X = model_featureSelection.fit_transform(Classifier_Train_X,Classifier_Train_Y)
    print('After Num features=',Classifier_Train_X.shape[1])

    
    
    if(save_model==True):
        Classifier=model
        Classifier.fit(Classifier_Train_X, Classifier_Train_Y)
        filename = 'taskB1/'+model_name+'_task_2.joblib.pkl'
        joblib.dump(Classifier, filename, compress=9)
        filename1 = 'taskB1/'+model_name+'_select_features_task2.joblib.pkl'
        joblib.dump(model_featureSelection, filename1, compress=9)
    else:
        from sklearn.model_selection import KFold
        kf = skf(n_splits=10)
        y_total_preds=[] 
        y_total=[]
        count=0

        for train_index, test_index in kf.split(Classifier_Train_X,Classifier_Train_Y):
            X_train, X_test = Classifier_Train_X[train_index], Classifier_Train_X[test_index]
            y_train, y_test = Classifier_Train_Y[train_index], Classifier_Train_Y[test_index]
            classifier=model 
            classifier.fit(X_train,y_train)
            y_preds = classifier.predict(X_test)
            for ele in y_test:
                y_total.append(ele)
            for ele in y_preds:
                if(model_name=='Catboost'):
                    y_total_preds.append(ele[0])
                else:
                    y_total_preds.append(ele)
                    
            y_pred_train = classifier.predict(X_train)
            count=count+1       
            print('accuracy_train:',accuracy_score(y_train, y_pred_train),'accuracy_test:',accuracy_score(y_test, y_preds))
            print('TRAINING:')
            print(classification_report( y_train, y_pred_train ))
            print("TESTING:")
            print(classification_report( y_test, y_preds ))

        report = classification_report( y_total, y_total_preds )
        cm=confusion_matrix(y_total, y_total_preds)
        plt=plot_confusion_matrix(cm,normalize= True,target_names = ['discredit','sexual_harassment','stereotype','dominance','derailing'],title = "Confusion Matrix")
        plt.savefig('task2'+model_name+'_'+img_name)
        print(model)
        print(report)
        print(accuracy_score(y_total, y_total_preds))
        df_result=pandas_classification_report(y_total,y_total_preds)
        df_result.to_csv('task2'+model_name+'_'+report_name,  sep=',')


In [19]:
classifier_model = 'gaussian'
feature_model = 'google_nopreprocess_tfidf_preprocess_embed'
img_name = 'cm.png'
report_name = 'report.csv'

In [20]:
data_name= 'pd_train_category'

if(data_name=='pd_train_binary'):
    X,y=get_feature(f_type=feature_model,data=pd_train_binary)
    label_map = {
             1: 1,
             0: 0
                 }
elif(data_name=='pd_train_category'):
    X,y=get_feature(f_type=feature_model,data=pd_train_category)
    label_map = {
            'discredit': 0,
            'sexual_harassment': 1,
            'stereotype': 2,
            'dominance': 3,
            'derailing': 4
        }
elif(data_name=='pd_train_target'):
    X,y=get_feature(f_type=feature_model,data=pd_train_target)
    label_map = {
             'active': 1,
             'passive': 0
         }

else:
    print('give correct data')
    

ending
google_nopreprocess_tfidf_preprocess_embed


In [21]:
model=get_model(m_type=classifier_model)

GaussianNB(priors=None)


In [26]:
#list_of_model = ['decision_tree_classifier', 'gaussian', 'logistic_regression', 'MLPClassifier', 'RandomForestClassifier',
#                 'SVC', 'Catboost', 'XGB_classifier']
list_of_model = [ 'Catboost','XGB_classifier']

for each_model in list_of_model:
    model=get_model(m_type=each_model)
    binny_classifier_run(X,y,model,each_model,label_map,img_name,report_name,save_model=True)


Before Num features= 15712 Counter({0: 1014, 1: 352, 2: 179, 3: 148, 4: 92})
After Num features= 277
0:	learn: -1.5955220	total: 118ms	remaining: 1m 57s
1:	learn: -1.5806177	total: 211ms	remaining: 1m 45s
2:	learn: -1.5668825	total: 316ms	remaining: 1m 44s
3:	learn: -1.5572618	total: 409ms	remaining: 1m 41s
4:	learn: -1.5432442	total: 501ms	remaining: 1m 39s
5:	learn: -1.5316033	total: 592ms	remaining: 1m 38s
6:	learn: -1.5220850	total: 672ms	remaining: 1m 35s
7:	learn: -1.5113581	total: 766ms	remaining: 1m 34s
8:	learn: -1.5012559	total: 851ms	remaining: 1m 33s
9:	learn: -1.4895076	total: 939ms	remaining: 1m 32s
10:	learn: -1.4823338	total: 1.02s	remaining: 1m 31s
11:	learn: -1.4721065	total: 1.11s	remaining: 1m 31s
12:	learn: -1.4629189	total: 1.2s	remaining: 1m 30s
13:	learn: -1.4527619	total: 1.27s	remaining: 1m 29s
14:	learn: -1.4419376	total: 1.37s	remaining: 1m 29s
15:	learn: -1.4322653	total: 1.45s	remaining: 1m 29s
16:	learn: -1.4240560	total: 1.53s	remaining: 1m 28s
17:	learn

153:	learn: -0.9084764	total: 13.1s	remaining: 1m 11s
154:	learn: -0.9064558	total: 13.1s	remaining: 1m 11s
155:	learn: -0.9047067	total: 13.2s	remaining: 1m 11s
156:	learn: -0.9024586	total: 13.3s	remaining: 1m 11s
157:	learn: -0.9000893	total: 13.4s	remaining: 1m 11s
158:	learn: -0.8977697	total: 13.5s	remaining: 1m 11s
159:	learn: -0.8960987	total: 13.6s	remaining: 1m 11s
160:	learn: -0.8938402	total: 13.7s	remaining: 1m 11s
161:	learn: -0.8904861	total: 13.8s	remaining: 1m 11s
162:	learn: -0.8888697	total: 13.8s	remaining: 1m 11s
163:	learn: -0.8859375	total: 13.9s	remaining: 1m 10s
164:	learn: -0.8844633	total: 14s	remaining: 1m 10s
165:	learn: -0.8825860	total: 14.1s	remaining: 1m 10s
166:	learn: -0.8808846	total: 14.2s	remaining: 1m 10s
167:	learn: -0.8794049	total: 14.2s	remaining: 1m 10s
168:	learn: -0.8778777	total: 14.3s	remaining: 1m 10s
169:	learn: -0.8763500	total: 14.4s	remaining: 1m 10s
170:	learn: -0.8748947	total: 14.5s	remaining: 1m 10s
171:	learn: -0.8739102	total: 

309:	learn: -0.6374366	total: 26.2s	remaining: 58.3s
310:	learn: -0.6361518	total: 26.3s	remaining: 58.2s
311:	learn: -0.6345903	total: 26.3s	remaining: 58.1s
312:	learn: -0.6330021	total: 26.4s	remaining: 58s
313:	learn: -0.6305330	total: 26.5s	remaining: 57.9s
314:	learn: -0.6284950	total: 26.6s	remaining: 57.8s
315:	learn: -0.6262111	total: 26.7s	remaining: 57.7s
316:	learn: -0.6250200	total: 26.8s	remaining: 57.7s
317:	learn: -0.6233500	total: 26.8s	remaining: 57.6s
318:	learn: -0.6222518	total: 26.9s	remaining: 57.5s
319:	learn: -0.6206391	total: 27s	remaining: 57.4s
320:	learn: -0.6190048	total: 27.1s	remaining: 57.3s
321:	learn: -0.6173044	total: 27.2s	remaining: 57.2s
322:	learn: -0.6160394	total: 27.3s	remaining: 57.1s
323:	learn: -0.6143378	total: 27.3s	remaining: 57s
324:	learn: -0.6121662	total: 27.4s	remaining: 57s
325:	learn: -0.6095717	total: 27.5s	remaining: 56.9s
326:	learn: -0.6071806	total: 27.6s	remaining: 56.8s
327:	learn: -0.6061632	total: 27.7s	remaining: 56.7s
3

465:	learn: -0.4461854	total: 38.9s	remaining: 44.5s
466:	learn: -0.4454733	total: 38.9s	remaining: 44.4s
467:	learn: -0.4441195	total: 39s	remaining: 44.4s
468:	learn: -0.4429415	total: 39.1s	remaining: 44.3s
469:	learn: -0.4419509	total: 39.2s	remaining: 44.2s
470:	learn: -0.4412139	total: 39.3s	remaining: 44.1s
471:	learn: -0.4403568	total: 39.4s	remaining: 44s
472:	learn: -0.4390677	total: 39.4s	remaining: 44s
473:	learn: -0.4386787	total: 39.5s	remaining: 43.9s
474:	learn: -0.4379790	total: 39.6s	remaining: 43.8s
475:	learn: -0.4373307	total: 39.7s	remaining: 43.7s
476:	learn: -0.4368731	total: 39.7s	remaining: 43.6s
477:	learn: -0.4356652	total: 39.8s	remaining: 43.5s
478:	learn: -0.4343828	total: 39.9s	remaining: 43.4s
479:	learn: -0.4334422	total: 40s	remaining: 43.3s
480:	learn: -0.4322428	total: 40.1s	remaining: 43.3s
481:	learn: -0.4317275	total: 40.2s	remaining: 43.2s
482:	learn: -0.4312098	total: 40.2s	remaining: 43.1s
483:	learn: -0.4306125	total: 40.3s	remaining: 43s
484

621:	learn: -0.3328437	total: 51.5s	remaining: 31.3s
622:	learn: -0.3322767	total: 51.5s	remaining: 31.2s
623:	learn: -0.3319827	total: 51.6s	remaining: 31.1s
624:	learn: -0.3313082	total: 51.7s	remaining: 31s
625:	learn: -0.3303172	total: 51.8s	remaining: 30.9s
626:	learn: -0.3296787	total: 51.9s	remaining: 30.9s
627:	learn: -0.3293320	total: 51.9s	remaining: 30.8s
628:	learn: -0.3286808	total: 52s	remaining: 30.7s
629:	learn: -0.3279204	total: 52.1s	remaining: 30.6s
630:	learn: -0.3276702	total: 52.2s	remaining: 30.5s
631:	learn: -0.3273713	total: 52.2s	remaining: 30.4s
632:	learn: -0.3267292	total: 52.3s	remaining: 30.3s
633:	learn: -0.3264252	total: 52.4s	remaining: 30.3s
634:	learn: -0.3262042	total: 52.5s	remaining: 30.2s
635:	learn: -0.3256284	total: 52.6s	remaining: 30.1s
636:	learn: -0.3252646	total: 52.6s	remaining: 30s
637:	learn: -0.3249139	total: 52.7s	remaining: 29.9s
638:	learn: -0.3243946	total: 52.8s	remaining: 29.8s
639:	learn: -0.3239507	total: 52.9s	remaining: 29.7s

780:	learn: -0.2609282	total: 1m 4s	remaining: 18s
781:	learn: -0.2607214	total: 1m 4s	remaining: 17.9s
782:	learn: -0.2604466	total: 1m 4s	remaining: 17.8s
783:	learn: -0.2600405	total: 1m 4s	remaining: 17.7s
784:	learn: -0.2593663	total: 1m 4s	remaining: 17.7s
785:	learn: -0.2589410	total: 1m 4s	remaining: 17.6s
786:	learn: -0.2582655	total: 1m 4s	remaining: 17.5s
787:	learn: -0.2580678	total: 1m 4s	remaining: 17.4s
788:	learn: -0.2579172	total: 1m 4s	remaining: 17.3s
789:	learn: -0.2573962	total: 1m 4s	remaining: 17.3s
790:	learn: -0.2572190	total: 1m 4s	remaining: 17.2s
791:	learn: -0.2567047	total: 1m 5s	remaining: 17.1s
792:	learn: -0.2563477	total: 1m 5s	remaining: 17s
793:	learn: -0.2559804	total: 1m 5s	remaining: 16.9s
794:	learn: -0.2554635	total: 1m 5s	remaining: 16.8s
795:	learn: -0.2551837	total: 1m 5s	remaining: 16.8s
796:	learn: -0.2549520	total: 1m 5s	remaining: 16.7s
797:	learn: -0.2545993	total: 1m 5s	remaining: 16.6s
798:	learn: -0.2543992	total: 1m 5s	remaining: 16.

936:	learn: -0.2097812	total: 1m 16s	remaining: 5.15s
937:	learn: -0.2094456	total: 1m 16s	remaining: 5.07s
938:	learn: -0.2091962	total: 1m 16s	remaining: 4.99s
939:	learn: -0.2089707	total: 1m 16s	remaining: 4.9s
940:	learn: -0.2085733	total: 1m 16s	remaining: 4.82s
941:	learn: -0.2079955	total: 1m 17s	remaining: 4.74s
942:	learn: -0.2078668	total: 1m 17s	remaining: 4.66s
943:	learn: -0.2075687	total: 1m 17s	remaining: 4.58s
944:	learn: -0.2074374	total: 1m 17s	remaining: 4.5s
945:	learn: -0.2068508	total: 1m 17s	remaining: 4.42s
946:	learn: -0.2066010	total: 1m 17s	remaining: 4.33s
947:	learn: -0.2064536	total: 1m 17s	remaining: 4.25s
948:	learn: -0.2061395	total: 1m 17s	remaining: 4.17s
949:	learn: -0.2058975	total: 1m 17s	remaining: 4.09s
950:	learn: -0.2057784	total: 1m 17s	remaining: 4s
951:	learn: -0.2053033	total: 1m 17s	remaining: 3.92s
952:	learn: -0.2051271	total: 1m 17s	remaining: 3.84s
953:	learn: -0.2050501	total: 1m 17s	remaining: 3.76s
954:	learn: -0.2046049	total: 1m 